In [1]:
cd ..

/home/arghya/sandbox/projects/champions_league_predictions


In [2]:
# import dependencies
import numpy as np
import pandas as pd
import csv
from sklearn.utils import shuffle

In [3]:
#import player stats and historical results for european clubs
data_path = 'data/'

#load player dataset

player_dataset=pd.read_csv(data_path+'CompleteDataset.csv',delimiter=',')
epl_clubs=['Manchester City', 'Liverpool', 'Tottenham Hotspur', 'Chelsea', 'Arsenal', 'Everton', 
           'Bournemouth', 'Manchester United', 'Leicester City','Brighton & Hove Albion', 'Watford', 
           'Wolverhampton Wanderers', 'West Ham United', 'Newcastle United', 'Crystal Palace',
           'Cardiff City', 'Huddersfield Town', 'Southampton', 'Burnley', 'Fulham']

#convert club names to lower case
for i in range(0,len(epl_clubs)):
    epl_clubs[i]=epl_clubs[i].lower()

#drop columns that are not required
player_dataset.drop(labels = ['Unnamed: 0', 'Age', 'Photo', 'Flag', 'Club Logo', 'Wage'], axis = 1, inplace = True)

#keeping clubs that are present in the premiere league
player_dataset['Club']=player_dataset['Club'].str.lower()
player_dataset = player_dataset[player_dataset['Club'].isin(epl_clubs)]

#Adding preferred position and remaining potential
player_dataset['Preferred Position'] = player_dataset['Preferred Positions'].str.split().str[0]
player_dataset['Remaining Potential'] = player_dataset['Potential'] - player_dataset['Overall']

# Loading and manipulating old league data
old_data = pd.read_csv(data_path+'merged_all.csv',delimiter=',')
old_data=old_data[old_data.FT!='FT']
old_data['Team 1']=old_data['Team 1'].str[:-7]
old_data['Team 2']=old_data['Team 2'].str[:-7]
old_data['home_score']=old_data['FT'].str[0]
old_data['away_score']=old_data['FT'].str[-1]
old_data['Team 1']=old_data['Team 1'].str.lower()
old_data['Team 2']=old_data['Team 2'].str.lower()
old_data['home_team']=old_data['Team 1']
old_data['away_team']=old_data['Team 2']
old_data.home_score=old_data.home_score.astype('float')
old_data.away_score=old_data.away_score.astype('float')

old_data.drop(labels=['Round','Date','HT','FT','Team 1','Team 2'],axis=1,inplace=True)
#saving processed fifa dataset with EPL clubs and prior match results

player_dataset.to_csv(data_path+'fifa_dataset_processed.tsv',sep='\t')
old_data.to_csv(data_path+'prior_fixtures.tsv',sep='\t')


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
mean_home_goals=old_data.groupby('home_team')['home_score'].mean().to_frame()
mean_away_goals=old_data.groupby('away_team')['away_score'].mean().to_frame()
df = pd.concat([mean_away_goals,mean_home_goals],axis=1)
df['advantage']=df.home_score - df.away_score
df.to_csv('data/home_advantage.tsv',sep='\t')
